# Building the UI
## Before we start (OPTIONAL)
Before we start, it might be a good idea to fork this repository. This is to ensure that we can easily get a link to your schema.

![fork](./img/fork.png)

Once you forked the repo, you can then clone your own repository.

## Introduction
In addition to the Neo4j console and the notebook earlier, we can provide develop a public facing website that we can share with the public without providing them write access to the neo4j database.

The UI provides single and two-term search where the latter uses allShortestPath to query all the shortest paths between two terms. To build the UI, we need to create a UI schema that is built using the data that was ingested in the graph db. This notebook automates the creation of this schema file.

In [1]:
import re
import json
from glob import glob
import pandas as pd

## Node Schema

```
{
    "node": "Gene",
    "example": [
        "COPB2",
        "SDF2L1"
    ],
    "display": [
        {
            "label": "Gene ID",
            "text": "${id}",
            "type": "text"
        },
        {
            "label": "label",
            "text": "${label}",
            "type": "text"
        },
        {
            "label": "uri",
            "text": "${uri}",
            "href": "${uri}",
            "type": "link"
        }
    ],
    "search": [
        "id",
        "label"
    ],
    "color": "#c5e1a5"
}

```
This is a node schema for the node "Gene". The example field will be displayed display as an example on the UI while the display field tells the UI what to display as a tooltip. Meanwhile, search tells the UI that we can search both id and label fields of the nodes. Node colors are automatically generated but can otherwise be set using the field color for more customization.

In [2]:
node_schema = []
node_pattern = "(?P<directory>.+)/(?P<label>.+)\.(?P<entity>.+)\.csv"
for filename in glob("csv/*.nodes.csv"):
    print(filename)
    match = re.match(node_pattern, filename).groupdict()
    label = match["label"].replace("_", " ")
    df = pd.read_csv(filename)
    display = []
    for i in df.columns:
        if i == "id":
            display.append({
                'label': '%s ID'%label,
                'text': "${%s}"%i,
                'type': 'text'
            })
        else:
            val = str(df.at[0, i])
            if val.startswith("http"):
                display.append({
                    'label': i,
                    'text': "${%s}"%i,
                    'href': "${%s}"%i,
                    'type': 'link'
                })
            else:
                display.append({
                    'label': i,
                    'text': "${%s}"%i,
                    'type': 'text'
                })
    examples = list(df.head(2).label)
    node = {
        "node": label,
        "example": examples,
        "display": display,
        "relation": [],
        "search": [
            "id",
            "label",
        ],
        # "color": "#c5e1a5"
    }
    node_schema.append(node)
        

csv/kinase.nodes.csv
csv/kinase_phosphosite.nodes.csv


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/bm/dzhmgjqn38j8l0wyy8chzy1m0000gn/T/ipykernel_59797/2230849790.py:2: SyntaxWarning: invalid escape sequence '\.'
  node_pattern = "(?P<directory>.+)/(?P<label>.+)\.(?P<entity>.+)\.csv"


In [15]:
node_pattern

'(?P<directory>.+)/(?P<label>.+)\\.(?P<entity>.+)\\.csv'

## Edge Schema
```
{
    "match": [
        "KEGG"
    ],
    "selected": true,
    "display": [
        {
            "label": "source",
            "text": "${source}",
            "type": "text"
        },
        {
            "label": "relation",
            "text": "${relation}",
            "type": "text"
        },
        {
            "label": "target",
            "text": "${target}",
            "type": "text"
        },
        {
            "label": "source label",
            "text": "${source_label}",
            "type": "text"
        },
        {
            "label": "target label",
            "text": "${target_label}",
            "type": "text"
        },
        {
            "label": "resource",
            "text": "${resource}",
            "href": "${resource}",
            "type": "link"
        }
    ]
}
```

This is the same with node schema with a few differences, first match allows you to match multiple edge types and selected is a field that allows us to select what edges are queried by default when doing two term searches.

In [3]:
edge_schema = []
edge_pattern = "(?P<directory>.+)/(?P<source_type>.+)\.(?P<relation>.+)\.(?P<target_type>.+)\.(?P<entity>.+)\.csv"

for filename in glob("csv/*.edges.csv"):
   print(filename)
   match = re.match(edge_pattern, filename).groupdict()
   entity = match["entity"]
   source_type = match["source_type"]
   relation = match["relation"].replace("_", " ")
   edge = {
      "match": [relation],
      "selected": True,
   }
   df = pd.read_csv(filename)
   display = []
   for i in df.columns:
      if i == "id":
         display.append({
               'label': '%s ID'%label,
               'text': "${%s}"%i,
               'type': 'text'
         })
      else:
         val = str(df.at[0, i])
         if val.startswith("http"):
               display.append({
                  'label': i.replace("_", " "),
                  'text': "${%s}"%i,
                  'href': "${%s}"%i,
                  'type': 'link'
               })
         else:
               display.append({
                  'label': i.replace("_", " "),
                  'text': "${%s}"%i,
                  'type': 'text'
               })
         edge["display"] = display
   edge_schema.append(edge)
         

csv/kinase_phosphosite.phosphosite.kinase.edges.csv
csv/kinase.phosphorylates.kinase_phosphosite.edges.csv


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/bm/dzhmgjqn38j8l0wyy8chzy1m0000gn/T/ipykernel_59797/1772761088.py:2: SyntaxWarning: invalid escape sequence '\.'
  edge_pattern = "(?P<directory>.+)/(?P<source_type>.+)\.(?P<relation>.+)\.(?P<target_type>.+)\.(?P<entity>.+)\.csv"


In [14]:
len(edge_schema)

2

## Additional schema

The next part are miscellaneous schemas that you can use to modify our UI. Notice the tabs field, this is where we define the pages in our knowledge graph as well as the components that you want to render.

Currently we have the following components available:
1. KnowledgeGraph: this renders the single and two term search components
2. Markdown: this is for rendering static markdown files, useful for tutorials
3. Enrichment: For enrichment page.
   
Each component takes specific props like init_function that initializes/prefetch the static props for the page. For this example, we only look at the KnowledgeGraph component. For more information on how to use the other components you can check: https://maayanlab.cloud/enrichr-kg/api/knowledge_graph/schema

In [4]:
# First you need to modify this based on your dataset. 
# This will be the initial query that the users will see when visiting your site
initial_query = {
	"start": "kinase",
	"start_term": "CDC7",
	"start_field": "source_label"
}

In [5]:
title = "KEA-KG"
icon = "https://minio.dev.maayanlab.cloud/enrichr-kg/enrichr-kg.png"
rest = {
    "header": {
        "divider": True,
        "header": "KEA-KG",
        "title": "KEA-KG",
        "counter": True,
        "counterTop": True,
        "background": {
            "backgroundColor": "#000",
            "contrastText": "#FFF"
        },
        "icon": {
            "src": "/hgrnchear_logo.png",
            "favicon": "/hgrnchear_logo.png",
            "faviconTitle": "KEA-KG",
            "alt": "KEA-KG",
            "key": "hGRN_ChEAR_icon",
            "width": 200,
            "height": 120,
            "avatar": False
        },
        "tabs": [
            {
                "endpoint": "/",
                "label": "Search",
                "type": "page",
                "component": "KnowledgeGraph",
                "props": {
                    "init_function": "initialize_kg",
                    "title": "Kinase-Kinase Regulatory Subnetworks",
                    "description": "Search for subnetworks within the KEA3 GRN by entering one or two kinases. The background GRN contains 42,322 signed and directed kinase-kinase regulatory relationships between 20,659 source and target human kinases and phosphosites.",
                    "initial_query": {
                        "start": "Kinase",
                        "start_term": "CDC7",
                        "start_field": "source_label"
                    },
                    "subheader": {
                        "url_field": "filter",
                        "query_field": "relation"
                    }
                }
            },
            {
                "endpoint":"/enrichment",
                "label":"Enrichment",
                "type":"page",
                "component":"KEA3Enrichment",
                "props": {
                    "init_function": "initialize_enrichment",
                    "title": "Kinase Enrichment Analysis with KEA3",
                    "description": "Enter a set of Entrez gene below to perform enrichment analysis using [KEA3](https://maayanlab.cloud/kea3/).",
                    "placeholder": "Enter newline separated gene sets",
                    "disableHeader": True,
                    "disableLibraryLimit": True,
                    "libraries": [
                        {
                            "name": "Integrated--meanRank",
                            "node": "Kinase"
                        }
                    ],
                    "sortLibraries": False,
                    "default_options": {
                        "min_lib": 3,
                        "libraries": [
                            {
                                "name": "Integrated--meanRank",
                                "limit": 10
                            }
                        ]
                    },
                    "subheader": {
                        "url_field": "q",
                        "query_field": "libraries"
                    }
                }
            },
        ]
    },
    "footer": {
        "style": {
            "background": "#1F3D5C",
            "color": "#FFF"
        },
        "layout": [
            [
                {
                    "component": "logo",
                    "props": {
                        "src": "/hgrnchear_white.png",
                        "alt": "hGRN_ChEAR_logo",
                        "title": "KEA-KG",
                        "color": "inherit",
                        "size": "medium"
                    }
                },
                {
                    "component": "github",
                    "props": {
                        "code": "https://github.com/MaayanLab/ChEA-KG-UI",
                        "issues": "https://github.com/MaayanLab/TranscriptionNetwork-Demo/issues"
                    }
                }
            ],
            [
                {
                    "component": "icon",
                    "props": {
                        "type": "icon",
                        "key": "lab_icon",
                        "src": "https://minio.dev.maayanlab.cloud/enrichr-kg/maayanlab.png",
                        "alt": "MaayanLab",
                        "href": "https://labs.icahn.mssm.edu/maayanlab/",
                        "height": 60,
                        "width": 150
                    }
                }
            ],
            [
                {
                    "component": "icon",
                    "props": {
                        "type": "icon",
                        "key": "center_icon",
                        "src": "https://minio.dev.maayanlab.cloud/enrichr-kg/bioinformatics.png",
                        "alt": "COB",
                        "href": "https://icahn.mssm.edu/research/bioinformatics",
                        "height": 60,
                        "width": 150
                    }
                }
            ],
            [
                {
                    "component": "license",
                    "props": {
                        "type": "icon",
                        "key": "center_icon",
                        "src": "https://s3.amazonaws.com/maayan-kg/chea-kg/gpl3.png",
                        "alt": "gpl3",
                        "href": "https://www.gnu.org/licenses/gpl-3.0.en.html",
                        "height": 60,
                        "width": 150
                    }
                }
            ]
        ],
        "footer_text": "<span style='color:white'>This website was prepared using KG UI. KG UI is being developed by the Ma'ayan Lab at Icahn School of Medicine and is available <a style='color: #81A1C1' href='https://github.com/MaayanLab/KnowledgeGraphUIDemo'>here</a>.</span>"
    },
	"ui_theme": "cfde_theme"
}

## Putting it all together

In [6]:
schema = {
    "nodes": node_schema,
    "edges": edge_schema,
    **rest
}

with open("../schema_sample.json", 'w') as o:
    o.write(json.dumps(schema, indent=4))

Once you have the schema you can upload it somewhere to obtain a link of the file. If you forked the repo you can commit it and push to your forked repository.

```
git add schema.json
git commit -m updated schema
git push
```

To get the link, go to schema.json on github and click raw:

![raw](./img/raw.png)
![raw_link](./img/raw_link.png)

Now let's clone the knowledge graph UI

```
git clone git@github.com:MaayanLab/Knowledge-Graph-UI.git
cd Knowledge-Graph-UI
git checkout next14
```

Now let's go and copy `.env.example`

```
cp .env.example .env
```

.env should look something like this

```
NEXT_PUBLIC_PREFIX=
NEXT_PUBLIC_NEO4J_DEV_URL=bolt://neo4j:7687
NEXT_PUBLIC_NEO4J_URL=bolt://neo4j:7687

NEXT_PUBLIC_NEO4J_USER=neo4j
NEXT_PUBLIC_NEO4J_NAME=kg

NEXT_PUBLIC_NEO4J_USER=neo4j
NEXT_PUBLIC_NEO4J_NAME=kg

NEXT_PUBLIC_NEO4J_PASSWORD=pass


NEXT_PUBLIC_DOMAINS=minio.dokku.maayanlab.cloud,github.com,minio.dev.maayanlab.cloud,s3.amazonaws.com,raw.githubusercontent.com
NEXT_PUBLIC_ENRICHR_URL=https://maayanlab.cloud/Enrichr
NEXT_PUBLIC_GENESHOT_URL=https://maayanlab.cloud/geneshot
NEXT_PUBLIC_HOST=http://localhost:3000
NEXT_PUBLIC_SCHEMA=https://raw.githubusercontent.com/MaayanLab/KnowledgeGraphDemo/main/schema.json
NEXT_PUBLIC_TURL=turlkey
NEXT_PUBLIC_TURL_URL=https://maayanlab.cloud/turl/api/register
APP_NAME=kg-demo
DOCKERHUB_NAME=maayanlab
```

Now go ahead and change `NEXT_PUBLIC_SCHEMA` to your link, you can also change `APP_NAME` and `DOCKERHUB_NAME` according to your dockerhub account

Run the following:
```
cd cli
python version-compose.py
cd ..
```
This adjusts your compose file based on your .env

## Ingestion
Now let's try ingesting data. Runa the following script

```
docker-compose up neo4j
# this is usually python ../src/import_csv.py ../notebooks/csv 
python /path/to/import_csv.py /path/to/csv
```
![ingestion](img/ingestion.png)


## Running it locally

You might need to install Node.js and NPM first https://docs.npmjs.com/downloading-and-installing-node-js-and-npm
```
npm i
npm run dev
```
go to http://localhost:3000


## Building and Running Docker Container

```
docker-compose build kg
docker-compose up
```
you can check http://localhost:3000 again

```
# push to dockerhub
docker-compose push kg
```